In [1]:
import numpy as np
from linearmodels.datasets import mroz, wage, birthweight, card, jobtraining
from statsmodels.api import add_constant
from mitools import reg
import pandas as pd
from linearmodels.iv import compare

# Instrumental Variables from Linearmodels

https://bashtage.github.io/linearmodels/iv/examples/basic-examples.html

## Basic Examples

These examples are based on **Chapter 15** of Introduction to Econometrics by Jeffrey Wooldridge and demonstrate the basic use of the IV estimators (primarily IV2SLS – the two-stage least squares estimator)

### Wages of Married Women

This first example examines the effect of education on the wages of women. Education is a classic endogenous variable since it has signaling value beyond the actual direct effect (among other reasons).

This first block imports the data and uses the `DESCR` attribute to describe the dataset. `add_constant` from statsmodels is used to add a variable named `const` to the DataFrame. Observations with missing values are dropped.

In [ ]:
print(mroz.DESCR)
data = mroz.load()
data = data.dropna()
data = add_constant(data, has_constant="add")

Since OLS is a special case of 2SLS, IV2SLS can be used to estimate a model using OLS by setting `endog` and `instruments` variables to `None`.

This first regression uses OLS to estimate the effect of education on the log of wage. It indicates that 1 year of education increases wage by 10%.

In [ ]:
res_ols = reg.IV2SLSModel(data, dependent_variable='lwage', independent_variables=["const", "educ"]).fit(
    cov_type="unadjusted"
)
print(res_ols)

Two-stage least squares is estimated “as-if” two regressions are run. Here the first stage regression where OLS is used to fit the value on the instrument (in this case the education of the subject”s father). The fitted value is saved for use later.

This first stage regression indicates that there is a strong relationship and the first stage easily passes the rule of thumb test where the F-statistic is at least 10.

In [ ]:
res_first = reg.IV2SLSModel(data, dependent_variable='educ', independent_variables=["const", "fatheduc"]).fit(
    cov_type="unadjusted"
)
print(res_first)
data["educ_hat"] = data.educ - res_first.resids

The second stage uses the instrument to fit the model. This model indicates a much lower effect of education.

In [ ]:
res_second = reg.IV2SLSModel(data, 
                             dependent_variable='lwage',
                             independent_variables=["const"], 
                             endogenous_variables=['educ'], 
                             instrument_variables=["fatheduc"]
                             ).fit(cov_type="unadjusted")
print(res_second)

The fitted value can be used with OLS to estimate the same parameters. Note that the other values reported such as t-statistics or $R^{2}$ are not correct.

The `compare` function is used to compare the different models. The 2SLS coefficient on education and the direct coefficient on educ_hat are identical.

In [ ]:
res_direct = reg.IV2SLSModel(data, dependent_variable="lwage", independent_variables=["const", "educ_hat"]).fit(
    cov_type="unadjusted"
)
print(compare({"OLS": res_ols, "2SLS": res_second, "Direct": res_direct}))

### Wages of Men

This next example examines the returns to education for men and uses the number of siblings as an instrument.

In [ ]:
men = wage.load()
print(wage.DESCR)
men = men[["educ", "wage", "sibs", "exper"]]
men['lwage'] = np.log(men.wage)
men = add_constant(men)
men = men.dropna()

In [ ]:
res_ols = reg.IV2SLSModel(men, dependent_variable='lwage', independent_variables=["const", "educ"]).fit(
    cov_type="unadjusted"
)
print(res_ols)

The first stage regressed the endogenous variable on the instrument. There is a strong, negative relationship here.

In [ ]:
res_first = reg.IV2SLSModel(men, dependent_variable='educ', independent_variables=["const", "sibs"]).fit(
    cov_type="unadjusted"
)
print(res_first.summary)

The second stage indicates a much strong relationship than the OLS estimate.

In [ ]:
res = reg.IV2SLSModel(men, 
                      dependent_variable='lwage', 
                      independent_variables=["const"], 
                      endogenous_variables=['educ'], 
                      instrument_variables=["sibs"]
                      ).fit(cov_type="unadjusted")
print(res.summary)


### Weighted IV

All IV estimator support weighted which extends the concept of WLS in an OLS model to IV estimation. The weights are applied to the dependent variables, the matrix of regressors (endogenous and exogenous) and the matrix of instrument (exogenous and instruments) which allows for GLS-type estimation. In particular, if the variance of model residuals was $\sigma^2$, then setting $\omega = 1/\sigma^2$ would produce GLS estimates.

This example shows how a feasible GLS-type of estimator could be used.

In [ ]:
res2 = res.resids**2
men['lres2'] = np.log(res2)
fgls_mod = reg.IV2SLSModel(men, 
                           dependent_variable='lres2', 
                           independent_variables=["const", "sibs", "exper"], 
                           )
fgls_res = fgls_mod.fit()
sigma2_hat = np.exp(np.log(res2) - fgls_res.resids)
print(fgls_res)


The variance of the squared residuals is not particularly well explained by the data, and so the GLS-type estimates and the usual IV estimates don”t differ by much.

In [ ]:
res_gls = reg.IV2SLSModel(men, 
                          dependent_variable='lwage', 
                          independent_variables=["const"], 
                          endogenous_variables=['educ'], 
                          instrument_variables=["sibs"], 
                          weights=1/sigma2_hat
                          ).fit(cov_type="unadjusted")
print(res_gls)

### Smoking and birth weight

This example examines the effect of smoking on the birth weight of babies.

In [ ]:
data = birthweight.load()
print(birthweight.DESCR)
data = add_constant(data)
data['lbwght'] = np.log(data.bwght)

The first stage regresses the number of packs smoked on the cigarette price. There is a very weak relationship – so weak that this is an example of a _weak instrument.

In [ ]:
res = reg.IV2SLSModel(data, 
                      dependent_variable='packs', 
                      independent_variables=["const", "cigprice"]
                      ).fit(cov_type="unadjusted")
print(res)

Despite the weak relationship between the price and the number of pack smoked, the 2SLS can be estimated, although substantial caution is warranted to interpret the results. Note the very negative $R^2$ (-150%).

In [ ]:
res = reg.IV2SLSModel(data, 
                      dependent_variable='lbwght', 
                      independent_variables=["const"], 
                      endogenous_variables=['packs'], 
                      instrument_variables=["cigprice"]
                      ).fit(cov_type="unadjusted")
print(res)

### Proximity and education

This next example uses a well-known dataset that uses proximity to a 4 year college as an instrument for education.

In [ ]:
data = card.load()
print(card.DESCR)
data = add_constant(data)

Lists are used to hold the groups of variable in this large model and missing values are dropped.



In [17]:
dep = ["wage"]
endog = ["educ"]
exog = [
    "const",
    "exper",
    "expersq",
    "black",
    "smsa",
    "south",
    "smsa66",
    "reg662",
    "reg663",
    "reg664",
    "reg665",
    "reg666",
    "reg667",
    "reg668",
    "reg669",
]
instr = ["nearc4"]
data = data[dep + exog + endog + instr].dropna()
data['lwage'] = np.log(data.wage)

The first stage estimate shows a very large F-statistic. Note that when there are many exogenous variables the results cannot be directly interpreted. It is better to use the `first_stage` information from a 2SLS estimate.

In [ ]:
res = reg.IV2SLSModel(data, dependent_variable='educ', independent_variables=instr + exog).fit()
print(res)

The OLS estimate indicates an increase of 7% for a year of education.

In [ ]:
res = reg.IV2SLSModel(data, 
                      dependent_variable='lwage', 
                      independent_variables=exog + endog, 
                      ).fit()
print(res)

The 2SLS estimate of the effect is nearly double. However, there is some reason to be concerned about the strength of the instrument despite the F-statistic in the first stage regression.

In [ ]:
res_2sls = reg.IV2SLSModel(data, 
                      dependent_variable='lwage', 
                      independent_variables=exog,
                      endogenous_variables=endog, 
                      instrument_variables=instr
                      ).fit()
print(res_2sls)

The property `first_stage` can be used to show a large set of first stage diagnostics. These results show a much lower partial $R^2$ that has measures the unique effect of the instrument on the endogenous controlling for the exogenous regressors. This is much smaller than the naive first stage $R^2$ of 47%. The partial F-statistic is also much smaller, although it technically over the rule-of-thumb of 10 for a single instrument.

The instrument is a dummy variable and being close to a 4 year college is only worth 0.3 years of education on average.

In [ ]:
print(res_2sls.first_stage)

### Formula interface

This model was large and so it might be simpler to use a formula. While formulas are discussed in detail in another notebook, they use the formula language of formulaic with an augmentation to specify the endogenous and instrumental variables. The generic form is
```
dependent ~ exogenous + endogenous ~ instruments
```
where each block can contain multiple variables.

Here the model is compared to the direct parameterization using DataFrames by differencing the $R^2$.

In [ ]:
formula = (
    "np.log(wage) ~ 1 + exper + expersq + black + smsa + south + smsa66 + reg662 + reg663 + reg664 + "
    "reg665 + reg666 + reg667 + reg668 + reg669 + [educ ~ nearc4]"
)
model = reg.IV2SLSModel(data, formula=formula)
res_formula = model.fit(cov_type="unadjusted")
print(round(res_formula.rsquared - res_2sls.rsquared, 10))

### Categorical Variables

pandas `categoricals` are automatically treated as factors and expanded to dummies. The first is always dropped. This next block constructs a categorical from the region dummies and then uses it instead of the individual dummies. The model is identical.

In [ ]:
data["reg"] = "661"  # The default region, which was omitted
for i in range(2, 10):
    region = "reg66" + str(i)
    data.loc[data[region] == 1, "reg"] = region[3:]
data["reg"] = data["reg"].astype("category")
data.describe()
res_cat = reg.IV2SLSModel(data, 
                          dependent_variable='lwage', 
                          independent_variables=["const", "exper", "expersq", "black", "smsa", "south", "smsa66", "reg"],
                          endogenous_variables=["educ"],
                          instrument_variables=["nearc4"]
                          ).fit()
print(res_cat)

### Post-estimation diagnostics

Common post-estimation diagnostics are to test the assumption of endogeneity and to examine if instruments are valid (when there are more instruments than endogenous variables).

In [24]:
data = mroz.load()
data = data.dropna()
data = add_constant(data, has_constant="add")
data["lnwage"] = np.log(data.wage)
dep = "lnwage"
exog = ["const", "exper", "expersq"]
endog = ["educ"]
instr = ["fatheduc", "motheduc"]

The first step is to fit the model using 2SLS.

In [ ]:
res = reg.IV2SLSModel(data, 
                      dependent_variable=dep, 
                      independent_variables=exog,
                      endogenous_variables=endog, 
                      instrument_variables=instr
                      ).fit(cov_type="unadjusted")
print(res)


Wooldridge”s regression test of exogeneity uses regression residual where the endogenous variables are regressed on the exogenous and the instrument to test for endogeneity. IF the endogenous variable is actually exogenous these residuals should not be correlated with the variable of interest.

In [ ]:
res.wooldridge_regression

This test can be easily implemented using two regression. The first one constructs the residuals and the second re-fits the model using 2SLS but including the residuals.

Note that the p-value of the t-state on `residuals` is the same as the P-value of the previous test – this is not an accident.

In [ ]:
v = reg.IV2SLSModel(data,
                    dependent_variable=endog[0],
                    independent_variables=exog + instr,
                    ).fit().resids
data['v'] = v
res_direct = reg.IV2SLSModel(data, 
                              dependent_variable=dep, 
                              independent_variables=["v"] + exog, 
                              endogenous_variables=endog, 
                              instrument_variables=instr
                              ).fit(cov_type="unadjusted")
print(res_direct)

Since this regression has two instrument it is possible to test for overidentification. Wooldridge”s overidentification test uses a regression to test whether the 2SLS residuals are uncorrelated with the instruments, which should be the case if the model is correct and the instruments are not needed in the original model.

In [ ]:
res.wooldridge_overid

A naive version of this test can be directly implemented. This direct implementation is different from the formal test but would be consistent if the model was overidentified.

In [29]:
data['u'] = res.resids
res = reg.IV2SLSModel(data, 
                              dependent_variable="u", 
                              independent_variables=["exper", "expersq"] + instr, 
                              ).fit()

The test is $n \times R^2$, and has the same $\chi^2$ distribution. The test statistic is slightly smaller but the conclusions are the same.


In [ ]:
res.nobs * res.rsquared

Husband”s education can be used as an additional instrument, and its validity tested.

In [31]:
instr = ["fatheduc", "motheduc", "huseduc"]
res = reg.IV2SLSModel(data, 
                      dependent_variable=dep, 
                      independent_variables=exog,
                      endogenous_variables=endog, 
                      instrument_variables=instr
                      ).fit(cov_type="unadjusted")

Testing overidentification does not indicate any difference from the previous result.

In [ ]:
res.wooldridge_overid

Directly testing using two regression would reach the same conclusion.

In [ ]:
data['u'] = res.resids
res = reg.IV2SLSModel(data, 
                    dependent_variable="u", 
                    independent_variables=["exper", "expersq"] + instr, 
                    ).fit()
res.nobs * res.rsquared

### Panel IV

Instrumental variable regression can also be used with panel data. This example makes use of first differences to eliminate a year-specific effect and then uses




In [ ]:
data = jobtraining.load()
print(jobtraining.DESCR)
data.head()
data = data.where(data.year.isin((1987, 1988)))
data = data.dropna(how="all", axis=0).sort_values(["fcode", "year"])
print(data.describe())
data = data.set_index("fcode")
data = data[["year", "hrsemp", "grant", "scrap", "lscrap"]]

In [ ]:
deltas = data.loc[data.year == 1988] - data.loc[data.year == 1987]
deltas = add_constant(deltas, has_constant="add")
deltas = deltas.dropna()
print(deltas.describe())

The first stage indicates a relatively strong relationship between grant and the number of hours employed. Note that grant is a dummy and so the coefficient is just the difference in means.

In [ ]:
model = reg.IV2SLSModel(deltas, dependent_variable='hrsemp', independent_variables=["const", "grant"])
print(model.fit(cov_type="unadjusted"))

Here a formula is used to specify the model since it is cleaner. Note that the [] contains the endogenous variables and instruments.

In [ ]:
model = reg.IV2SLSModel(deltas, formula="lscrap ~ 1 + [hrsemp ~ grant]")
res_iv = model.fit(cov_type="unadjusted")
print(res_iv)

The 2SLS estimate is nearly twice as large as the OLS estimate and is slightly more significant.

In [ ]:
res_ols = reg.IV2SLSModel(deltas, formula="lscrap ~ 1 + hrsemp").fit(cov_type="unadjusted")
print(compare({"Panel OLS": res_ols, "Panel IV": res_iv}))

In [ ]:
model = reg.IV2SLSModel(deltas, formula="lscrap ~ 1 + [hrsemp ~ grant]")
res_iv = model.fit(cov_type="unadjusted")
n = deltas.shape[0]
pred_exog = pd.DataFrame(np.ones((n, 1)), index=deltas.index)
res_iv.predict(exog=pred_exog, endog=deltas[["hrsemp"]])

***